# Proyecto: Detección de Fallas en Equipos Eléctricos (Colab)

**Autor:** Mauricio Gutiérrez

**Objetivo:** Notebook listo para ejecutar en Google Colab. Genera dataset sintético, realiza EDA, detecta anomalías con Isolation Forest y exporta un reporte ejecutivo en PDF.

## Instrucciones
1. Ejecutá las celdas en orden en Google Colab.
2. Si necesitás instalar dependencias, ejecutá la celda de instalación.
3. Al final se generará un archivo PDF (`Reporte_Ejecutivo_Mantenimiento_Predictivo.pdf`) en el entorno, que podés descargar.

> En Colab: `Archivo > Descargar .ipynb` para guardar el notebook localmente.

In [ ]:
# Celda 1 - Instalar dependencias (ejecutar solo si es necesario en Colab)
!pip install scikit-learn reportlab imbalanced-learn matplotlib pandas nbformat


In [ ]:
# Celda 2 - Importar librerías
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.metrics import confusion_matrix, classification_report
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image, Table, TableStyle
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib import colors
import os

print('Librerías cargadas correctamente')

In [ ]:
# Celda 3 - Generar dataset sintético (mejorado)
np.random.seed(42)
n_samples = 2000
time = pd.date_range(start='2023-01-01', periods=n_samples, freq='H')

temperature = 70 + 10 * np.sin(np.linspace(0, 20, n_samples)) + np.random.normal(0, 1, n_samples)
vibration = 0.02 * np.random.randn(n_samples) + 0.05
pressure = 30 + 2 * np.cos(np.linspace(0, 15, n_samples)) + np.random.normal(0, 0.5, n_samples)

anomaly_indices = np.random.choice(n_samples, size=40, replace=False)
temperature[anomaly_indices[:15]] += np.random.uniform(15, 25, 15)
vibration[anomaly_indices[15:30]] += np.random.uniform(0.2, 0.5, 15)
pressure[anomaly_indices[30:]] -= np.random.uniform(5, 10, 10)

labels = np.zeros(n_samples)
labels[anomaly_indices] = 1

df = pd.DataFrame({
    'timestamp': time,
    'temperature': temperature,
    'vibration': vibration,
    'pressure': pressure,
    'anomaly': labels
})

# Guardar CSV
os.makedirs('/content/drive/MyDrive/pasantiagruposie', exist_ok=True)
csv_path = '/content/drive/MyDrive/pasantiagruposie/machine_sensors_anomalies.csv'
df.to_csv(csv_path, index=False)

df.head()

In [ ]:
# Celda 4 - Exploración rápida (EDA)
print('Registros:', len(df))
print(df.describe())

plt.figure(figsize=(12,4))
plt.plot(df['timestamp'], df['temperature'], label='Temperatura')
plt.scatter(df['timestamp'][df['anomaly']==1], df['temperature'][df['anomaly']==1], color='r', s=10, label='Anomalía (sintética)')
plt.title('Temperatura vs Tiempo')
plt.xlabel('Tiempo')
plt.ylabel('Temperatura (°C)')
plt.legend()
plt.show()

In [ ]:
# Celda 5 - Preprocesamiento y modelo (Isolation Forest)
X = df[['temperature','vibration','pressure']].values
y_true = df['anomaly'].astype(int).values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

isof = IsolationForest(contamination=0.02, random_state=42)
isof.fit(X_scaled)
y_pred = isof.predict(X_scaled)
y_pred = np.where(y_pred == -1, 1, 0)

cm = confusion_matrix(y_true, y_pred)
print('Matriz de confusión:\n', cm)
print('\nReporte de clasificación:\n')
print(classification_report(y_true, y_pred, target_names=['Normal','Anomalía']))

In [ ]:
# Celda 6 - Guardar figuras para el reporte
fig1_path = '/content/drive/MyDrive/pasantiagruposie/fig_temperatura_tiempo.png'
plt.figure(figsize=(10,4))
plt.plot(df['timestamp'], df['temperature'])
plt.scatter(df['timestamp'].iloc[np.where(y_pred==1)[0]], df['temperature'].iloc[np.where(y_pred==1)[0]], s=10, color='r')
plt.title('Temperatura vs Tiempo (anomalías detectadas)')
plt.xlabel('Tiempo')
plt.ylabel('Temperatura (°C)')
plt.tight_layout()
plt.savefig(fig1_path, dpi=150)
plt.close()

# Matriz de confusión
fig2_path = '/content/drive/MyDrive/pasantiagruposie/fig_matriz_confusion.png'
plt.figure()
plt.imshow(cm, interpolation='nearest')
plt.title('Matriz de Confusión')
plt.colorbar()
plt.xticks([0,1], ['Normal','Anomalía'])
plt.yticks([0,1], ['Normal','Anomalía'])
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, format(cm[i, j], 'd'), ha='center', va='center')
plt.ylabel('Real')
plt.xlabel('Predicho')
plt.tight_layout()
plt.savefig(fig2_path, dpi=150)
plt.close()

print('Figuras guardadas en drive.')

In [ ]:
# Celda 7 - Generar PDF ejecutivo (Reporte)
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image, Table, TableStyle
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib import colors

pdf_path = '/content/drive/MyDrive/pasantiagruposie/Reporte_Ejecutivo_Mantenimiento_Predictivo.pdf'
doc = SimpleDocTemplate(pdf_path, pagesize=A4)
styles = getSampleStyleSheet()
story = []

story.append(Paragraph('<b>Reporte Ejecutivo</b>', styles['Title']))
story.append(Paragraph('Predicción de Fallas en Equipos Eléctricos (Prototipo de Mantenimiento Predictivo)', styles['Heading2']))
story.append(Spacer(1, 12))
story.append(Paragraph('Autor: Mauricio Gutiérrez', styles['Normal']))
story.append(Paragraph('Fecha: Generado en Google Colab', styles['Normal']))
story.append(Spacer(1, 12))

intro = (
    'Este prototipo demuestra cómo aplicar Inteligencia Artificial para detectar anomalías en señales de sensores '
    '(temperatura, vibración y presión) de equipos eléctricos. El objetivo es anticipar fallas potenciales, reducir '
    'paradas no programadas y mejorar la seguridad y la eficiencia operativa.'
)
story.append(Paragraph('<b>1. Introducción</b>', styles['Heading2']))
story.append(Paragraph(intro, styles['Normal']))
story.append(Spacer(1, 12))

metodologia = (
    'Se utilizó un conjunto de datos sintético que simula comportamientos normales y anomalías realistas (sobrecalentamiento, '
    'vibraciones anómalas y caídas de presión). Se entrenó un modelo Isolation Forest sobre variables normalizadas '
    'para identificar observaciones atípicas. El flujo es: sensores → preprocesamiento → detección de anomalías → alertas.'
)
story.append(Paragraph('<b>2. Metodología</b>', styles['Heading2']))
story.append(Paragraph(metodologia, styles['Normal']))
story.append(Spacer(1, 12))

# Insertar figura 1
story.append(Paragraph('<b>Figura 1.</b> Temperatura vs Tiempo con anomalías detectadas.', styles['Italic']))
story.append(Image(fig1_path, width=500, height=250))
story.append(Spacer(1, 12))

# Tabla matriz de confusión
story.append(Paragraph('<b>3. Resultados clave</b>', styles['Heading2']))
cm_table = [['', 'Pred: Normal', 'Pred: Anomalía'], ['Real: Normal', str(cm[0,0]), str(cm[0,1])], ['Real: Anomalía', str(cm[1,0]), str(cm[1,1])]]
table = Table(cm_table, colWidths=[100, 120, 120])
table.setStyle(TableStyle([('BACKGROUND', (0,0), (-1,0), colors.lightgrey), ('BOX', (0,0), (-1,-1), 0.5, colors.black), ('GRID', (0,0), (-1,-1), 0.25, colors.grey), ('ALIGN', (1,1), (-1,-1), 'CENTER')]))
story.append(table)
story.append(Spacer(1, 12))

story.append(Paragraph('<b>Figura 2.</b> Matriz de confusión del modelo.', styles['Italic']))
story.append(Image(fig2_path, width=400, height=300))
story.append(Spacer(1, 12))

report_text = classification_report(y_true, y_pred, target_names=['Normal','Anomalía'])
story.append(Paragraph('<b>4. Métricas (resumen)</b>', styles['Heading2']))
story.append(Paragraph('<pre>' + report_text + '</pre>', styles['Code']))
story.append(Spacer(1, 12))

recs = ['Integrar el modelo a un sistema de monitoreo (SCADA/EMS) para alertas tempranas.', 'Configurar umbrales y reglas de acción para mantenimiento preventivo.', 'Iterar el modelo con datos reales de tableros y transformadores (mejorará precisión).', 'Agregar sensores (si es necesario) para mejorar cobertura y confiabilidad.']
story.append(Paragraph('<b>5. Recomendaciones</b>', styles['Heading2']))
for r in recs:
    story.append(Paragraph(f'• {r}', styles['Normal']))

story.append(Spacer(1, 18))
story.append(Paragraph('Este reporte resume el valor potencial del mantenimiento predictivo con IA en entornos eléctricos industriales.', styles['Italic']))

doc.build(story)
print('PDF generado en:', pdf_path)


## Fin

Al ejecutar todas las celdas, encontrarás en tu Google Drive (carpeta `MyDrive/pasantiagruposie`) los archivos:
- `machine_sensors_anomalies.csv`
- `fig_temperatura_tiempo.png`
- `fig_matriz_confusion.png`
- `Reporte_Ejecutivo_Mantenimiento_Predictivo.pdf`

> En Colab: `Files` → `Download` para bajar los archivos, o descargarlos directamente desde Google Drive.